<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/RDWH_MySQL_Remote_Shell_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

##Purpose
To access MySQL at a remote server using credentials

#Remote Server
We use a remote MySQL server, for example, at <br>
http://remotemysql.com or <br>
http://clever-cloud.com

#Architecture
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/MySQLColab.png)<br>

In [1]:
!apt update > /dev/null
!apt install -y  mysql-server > /dev/null
!mysql --version
#!/etc/init.d/mysql restart -- no need to start local server





mysql  Ver 8.0.33-0ubuntu0.22.04.4 for Linux on x86_64 ((Ubuntu))


# Set Credentials

In [ ]:
# If this Notebook is not to be shared with others
# Then place your credentials here and uncomment the lines

#hostName = 'remotemysql.com'   # uncomment this line and place your own credentials here
#userName = 'yourusername'      # uncomment this line and place your own credentials here
#passWord = 'yourpassword'      # uncomment this line and place your own credentials here
#dbName =  'yourdbname'         # uncomment this line and place your own credentials here

#print(hostName,userName,dbName)
#
# If you are using the commands in this cell, then ignore the next three cells and to Install and Test

## Collect credentials from Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#
# copy credentials file
#!ls /content/drive/'My Drive'/Praxis/WebCredentials
!cp /content/drive/'My Drive'/Praxis/WebCredentials/cleverCloud.py credentials.py
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/db4freeCredentials.py credentials.py
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/cleverCloud.py credentials.py

# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM
# https://drive.google.com/drive/u/0/folders/1-EKWXDjIO_wOrNR6bR0U9IXbcFh31AMT
#from sqlCredentials import hostName,userName,passWord,dbName
from credentials import hostName,userName,passWord,dbName

print(hostName,userName,dbName)

Mounted at /content/drive
bq3ojarbqgmkc7prtopv-mysql.services.clever-cloud.com uxnhrnbgcu47thlz bq3ojarbqgmkc7prtopv


In [3]:
#NOTE SECURITY WARNING
#Passing passwords in the command line is inherently risky and SHOULD NOT BE USED for any non-trivial work
#---------------------------------------------------------------------
!mysql -h $hostName -P 3306 -u $userName -p$passWord $dbName -e 'SELECT VERSION()'
!mysql -h $hostName -P 3306 -u $userName -p$passWord $dbName -e 'show tables'
#!echo hostname $hostName
# --------------------------------------------------------------

mysql: [Warning] Using a password on the command line interface can be insecure.
+-----------+
| VERSION() |
+-----------+
| 8.0.22-13 |
+-----------+
mysql: [Warning] Using a password on the command line interface can be insecure.
+--------------------------------+
| Tables_in_bq3ojarbqgmkc7prtopv |
+--------------------------------+
| offices                        |
+--------------------------------+


##Install & Test Client

In [4]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb
from sqlalchemy import create_engine, text

In [5]:
#connect_string = 'mysql://root@localhost/praxisDB'
connect_string = 'mysql://'+userName+':'+passWord+'@'+hostName+'/'+dbName
#print(connect_string)
engine = create_engine(connect_string)
DBConn = engine.connect()

In [6]:
#DBConn = MySQLdb.connect(host=hostName,port=3306,user=userName,passwd=passWord,db=dbName)
DBConn = engine.connect()
#cursor = DBConn.cursor()
# execute SQL query using execute() method.
#cursor.execute("SELECT VERSION()")
cursor = DBConn.execute(text("SELECT VERSION()"))

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print ("Database version ", data)
# disconnect from server
DBConn.close()

Database version  ('8.0.22-13',)


## Define Pandas Functions

In [7]:
#To run any non-SELECT SQL command
def runCMD (DDL):
    #DBConn= MySQLdb.connect(host=hostName,port=3306,user=userName,passwd=passWord,db=dbName)
    DBConn = engine.connect()
    #myCursor = DBConn.cursor()
    #retcode = myCursor.execute(DDL)
    retcode = DBConn.execute(text(DDL))
    DBConn.commit()
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (CMD):
    #DBConn= MySQLdb.connect(host=hostName,port=3306,user=userName,passwd=passWord,db=dbName)
    DBConn = engine.connect()
    df_mysql = pd.read_sql(text(CMD), con=DBConn)
    DBConn.close()
    return df_mysql

#Create & Load Tables

In [8]:
runCMD("DROP TABLE IF EXISTS Emp;")
runCMD("CREATE TABLE IF NOT EXISTS Emp ( \
  EmpID char(6)  NOT NULL, \
  LastName varchar(50) , \
  FirstName varchar(50) , \
  JobDesc varchar(50) , \
  JoinDate date NOT NULL, \
  Salary int(11) , \
  Comm float , \
  DeptID char(2) \
) ;")

In [9]:
runCMD("INSERT INTO Emp (EmpID, LastName, FirstName, JobDesc, JoinDate, Salary, Comm, DeptID) \
    VALUES \
	('742866', 'Bacchan', 'Amitabh', 'Executive', '2003-03-10', 50000, 0.1, '10'), \
	('349870', 'Mukherjee', 'Rani', 'Manager', '2005-05-04', 25000, 0.06, '40'), \
	('865477', 'Dikshit', 'Madhuri', 'Clerk', '2002-04-04', 10000, 0.02, '20'), \
	('239456', 'Khan', 'Shahrukh', 'Manager', '2004-01-03', 30000, 0.07, '20'), \
	('897889', 'Sehwag', 'Virender', 'Cus_Rep', '2005-01-02', 15000, 0.05, '20'), \
	('123980', 'Dhoni', 'Mahender', 'Clerk', '2004-10-09', 9000, 0.02, '40'), \
	('822134', 'Dravid', 'Rahul', 'Sr Manager', '2000-06-04', 40000, 0.08, '30'), \
	('997445', 'Dalmia', 'Jagmohan', 'Clerk', '2001-07-01', 12000, 0.02, '30'), \
	('989007', 'Ganguly', 'Sourav', 'Cus_Rep', '2002-01-01', 20000, 0.03, '40'), \
	('299034', 'Ganesan', 'Rekha', 'Director', '2002-10-10', 60000, 0.11, '10'), \
	('546223', 'Karthikeyan', 'Narayan', 'Secretary', '2005-12-04', 40000, 0.09, '10'), \
	('223112', 'Mirza', 'Sania', 'Cus_Rep', '2001-11-19', 25000, 0.04, '30');"
    )

In [10]:
runCMD("DROP TABLE IF EXISTS Dept;")
runCMD("CREATE TABLE Dept ( \
  DeptID char(2)  NOT NULL, \
  DeptName varchar(50) , \
  ManagerID char(6) , \
  Location varchar(50) \
 );")

In [11]:
runCMD("INSERT INTO Dept (DeptID, DeptName, ManagerID, Location) VALUES \
	('10', 'Corporate', '299034', 'Calcutta'), \
	('20', 'Sales', '239456', 'Calcutta'), \
	('30', 'Accounts', '822134', 'Calcutta'), \
	('40', 'Production', '349870', 'Bombay');")

In [12]:
runSELECT('Select * from Emp;')

,EmpID,LastName,FirstName,JobDesc,JoinDate,Salary,Comm,DeptID
0,742866,Bacchan,Amitabh,Executive,2003-03-10,50000,0.10,10
1,349870,Mukherjee,Rani,Manager,2005-05-04,25000,0.06,40
2,865477,Dikshit,Madhuri,Clerk,2002-04-04,10000,0.02,20
3,239456,Khan,Shahrukh,Manager,2004-01-03,30000,0.07,20
4,897889,Sehwag,Virender,Cus_Rep,2005-01-02,15000,0.05,20
5,123980,Dhoni,Mahender,Clerk,2004-10-09,9000,0.02,40
6,822134,Dravid,Rahul,Sr Manager,2000-06-04,40000,0.08,30
7,997445,Dalmia,Jagmohan,Clerk,2001-07-01,12000,0.02,30
8,989007,Ganguly,Sourav,Cus_Rep,2002-01-01,20000,0.03,40
9,299034,Ganesan,Rekha,Director,2002-10-10,60000,0.11,10


In [13]:
runSELECT('Select * from Dept;')

,DeptID,DeptName,ManagerID,Location
0,10,Corporate,299034,Calcutta
1,20,Sales,239456,Calcutta
2,30,Accounts,822134,Calcutta
3,40,Production,349870,Bombay


##Clean UP

In [14]:
runCMD("DROP TABLE IF EXISTS Dept;")
runCMD("DROP TABLE IF EXISTS Emp;")


In [15]:
from datetime import datetime
import pytz
print('Executed on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Executed on  2023-07-28 08:48:37.617741+05:30
Python 3.10.6
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)